# Data Cleaning
Initial data cleaning and preprocessing for the project. This notebook is used to create the preprocessing pipeline in `pipeline.ipynb`. This used to clean the data for the exploratory data analysis and machine learning notebooks. It outputs a cleaned dataset to `CEIP_csv/cleaned.csv`.

#### Setup

In [1]:
# CORE
import pandas as pd
import os
import json
import numpy as np  # Numpy for numerical computations and array operations
import pandas as pd  # Pandas for data manipulation and analysis

# MACHINE LEARNING & STATISTICS 
import scipy.stats as stats  # SciPy for scientific computing and technical computing, including statistics
import sklearn as sk # Scikit-learn for machine learning and predictive modeling

# VISUALIZATION
import matplotlib.pyplot as plt  # Matplotlib for creating static, animated, and interactive visualizations
import seaborn as sns  # Seaborn for statistical data visualization built on top of Matplotlib
import plotly.express as px  # Plotly Express for creating interactive plots and charts
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# filepaths of all of the csv data files to be analyzed

# autonest_csv = "CEIP_csv/AutoNest.csv"
# autonest_strategy_csv = "CEIP_csv/AutoNestStrategy.csv"
# material_csv = "CEIP_csv/Material.csv"
# nest_csv = "CEIP_csv/Nest.csv"
# part_csv = "CEIP_csv/Part.csv"
# performance_csv = "CEIP_csv/Performance.csv"
training_csv = "../CEIP_csv/training_data.csv"

# read in all of these csv files as pandas dataframes

# autonest_df = pd.read_csv(autonest_csv)
# autonest_strategy_df = pd.read_csv(autonest_strategy_csv)
# material_df = pd.read_csv(material_csv)
# nest_df = pd.read_csv(nest_csv)
# part_df = pd.read_csv(part_csv)
# performance_df = pd.read_csv(performance_csv)
training_df = pd.read_csv(training_csv) # takes about 1 min to read 

## Data Cleaning

#### Data Previewing

In [3]:
# count the number of null values 
# training_df.isnull().sum() # --> RESULT: no null values in any column  

# list of columns
# column_list = list(training_df.columns)
column_list = ['ixJobSummary', 'ixNest', 'ixPart', 'dPartTrueArea', 'cRequired', 
               'cNested', 'ixMaterial', 'fExtShape', 'dExtArea', 'dExtBoundaryDist', 
               'dExtContainedDist', 'dLgIntArea', 'dLgIntBoundaryDist', 'dLgIntContainedDist', 
               'dLgExtConArea', 'dLgExtConBoundaryDist', 'dLgExtConContainedDist', 'cTimesCut', 
               'dNestingTime', 'fStrategies', 'dSheetLength', 'dSheetWidth', 'dSheetArea', 'dLengthUsed', 
               'dWidthUsed', 'dPartArea', 'calcUtil', 'ixAutoNestStrategy', 'fAllPartsNested']

# count the values for the ixMaterial column in the training_df 
# training_df.ixMaterial.value_counts()

# plot the distribution of the ixMaterial column in the training_df
# only plot the 10 most common values
# training_df.ixMaterial.value_counts().nlargest(10).plot(kind='bar', figsize=(10,5))

# count the number of unique values
# training_df.nunique(axis=0)

# ixJobSummary               224892
# ixNest                     224892
# ixPart                    4200357
# dPartTrueArea              984974
# cRequired                    1534
# cNested                      1654
# ixMaterial                   7316
# fExtShape                      52
# dExtArea                   825580
# dExtBoundaryDist           159907
# dExtContainedDist          526206
# dLgIntArea                  74924
# dLgIntBoundaryDist          35721
# dLgIntContainedDist         83215
# dLgExtConArea               71156
# dLgExtConBoundaryDist       24334
# dLgExtConContainedDist      68328
# cTimesCut                     176
# dNestingTime                23997
# fStrategies                   292
# dSheetLength                 2917
# dSheetWidth                  1937
# dSheetArea                  13217
# dLengthUsed                136950
# dWidthUsed                 112688
# dPartArea                  168268
# calcUtil                   172012
# ixAutoNestStrategy             13
# fAllPartsNested                 2
# dtype: int64

In [4]:
# above shows that there are only 4.2 million unique values for ixPart 
# this indicates that there are a lot of rows that are duplicated? 

# remove duplicates 
training_df = training_df.drop_duplicates()
training_df.shape # --> (5,762,622 rows, 29 columns)

(5762622, 29)

#### Re-encoding Variables
* One-hot encode all variables that are numerical but represent categories
* Replace ixMaterial and ixAutoNestStrategy with their appropriate values from the JSON file 
* Limit to only Materials that are steel

In [5]:
# replace ixMaterial column with the values from the MaterialTypes.json file 
# read in the json file as a pandas dataframe
material_types_df = pd.read_json('../CEIP_csv/MaterialTypes.json')

# create a dictionary of the material types and their corresponding values 
material_dict = material_types_df.to_dict()

# We need to make a new dictionary where the keys are the same but the values are the sNames
sName_dict = {k: v['sName'] for k, v in material_dict.items()}

# Now we use this dictionary to replace the ixMaterial values in training_df
training_df['ixMaterial'] = training_df['ixMaterial'].map(sName_dict)

# Rename the column
training_df = training_df.rename(columns={'ixMaterial': 'Material'})

In [6]:
# Count the different material types
material_counts = training_df['Material'].value_counts()
# results: 4,072,515 rows are MS, the next most is SS with 191,961 

# count the number of unique materials
nunique_materials = training_df.Material.nunique()
print(f'There are {nunique_materials} unique materials in the dataset.')

# Convert the Series to a DataFrame
material_counts_df = material_counts.reset_index()

# Rename the columns for clarity
material_counts_df.columns = ['Material', 'Count']

# Sort the DataFrame by the 'Count' column in descending order and take the top 30 rows
top_material_counts_df = material_counts_df.sort_values('Count', ascending=False).head(100)

# Create a bar chart for the top 30 materials
fig = px.bar(top_material_counts_df, x='Material', y='Count', title='Distribution of Top 100 Material Types')
fig.show()

There are 2117 unique materials in the dataset.


In [7]:
# remove all rows that do not have mild steel (ms) as the Material

# drop all the NA values from Material, Keep only the rows where Material contains 'ms'
training_df = training_df.dropna(subset=['Material'])
training_df = training_df[training_df['Material'] == 'ms']

# Drop the Material column from the dataframe - don't need it anymore 
encoded_df = training_df.drop(columns=['Material'])

In [14]:
# re-encode the fStrategies column 

# fStrategies = int 
# This is a bitmask comprised of the following nesting strategies:

# 0x00000000 = None
# 0x00000001 = Strategy1
# 0x00000002 = Strategy2	
# 0x00000004 = Strategy3	
# 0x00000008 = Strategy4	
# 0x00000010 = Strategy5	
# 0x00000020 = Strategy6	
# 0x00000040 = Strategy7	
# 0x00000080 = Strategy8	
# 0x00000100 = Strategy9	
# 0x00000200 = Strategy10	
# 0x00000400 = Block nesting	
# 0x00000800 = Block optimization 
# 0x00001000 = IntelliNest	
# 0x00002000 = IC Profile nesting	
# 0x00004000 = IC Pattern and fill
# 0x80000000 = Manual nesting	

# convert the bitmask values in the fStrategies to the string -> one-hot-encoding 
# result: 
# or is it good to use a bitmask? what problems might that lead to? is it good? 

# ?? what does it mean when it has a strategy in fStrategies AND one in ixAutoNestStrategy? 

# !! ask Mark - can we safely assume that if fStrategies uses IntelliNest, ixAutoNestStrategy is IntelliNest? 
# What additional info does ixAutoNestStrategy give us? 

# ?? Can a given Job use both manual nesting (chosen strategies in fStrategies) AND IntelliNest? 

# can also do a classifer that tries to predict the strategy used given the other columns (part characteristics)

# next step: can you make a multi-class classifier + regression model? 

# classify the strategy -> classify the strategy that yields the optimal util, GIVEN the part characteristics

In [8]:
# Load the dictionary from the JSON file
with open('../CEIP_csv/AutoNestStrategy.json', 'r') as f:
    autoneststrategy_dict = json.load(f)
    
#  Convert ixAutoNestStrategy to string
encoded_df['ixAutoNestStrategy'] = encoded_df['ixAutoNestStrategy'].astype(str)

# Replace the ixAutoNestStrategy values in the DataFrame
encoded_df['ixAutoNestStrategy'] = encoded_df['ixAutoNestStrategy'].map(autoneststrategy_dict)

# Count the different strategies
strategy_counts = encoded_df['ixAutoNestStrategy'].value_counts()

# Create a pie chart directly from the Series
fig = px.pie(strategy_counts, values=strategy_counts.values, names=strategy_counts.index, 
             title='Distribution of Strategies Used in AutoNest')
fig.show()

In [9]:
# one hot encoding for the AutoNestStrategy to convert categorical from numeric 
# One-hot encode ixAutoNestStrategy
encoded_df = pd.get_dummies(encoded_df, columns=['ixAutoNestStrategy'], prefix='')

# Prepare a dictionary for renaming the columns
rename_dict = {f'_{k}': v for k, v in autoneststrategy_dict.items()}

# Rename the columns
encoded_df.rename(columns=rename_dict, inplace=True)

encoded_df.head()

,ixJobSummary,ixNest,ixPart,dPartTrueArea,cRequired,cNested,fExtShape,dExtArea,dExtBoundaryDist,dExtContainedDist,...,_Strategy_1,_Strategy_10,_Strategy_2,_Strategy_3,_Strategy_4,_Strategy_5,_Strategy_6,_Strategy_7,_Strategy_8,_Strategy_9
0,304409,746165,2377314,151.1253,5,5,0,151.1253,4.7891,15.8795,...,0,0,0,0,1,0,0,0,0,0
1,304409,746165,2377315,49.5807,10,10,0,49.5807,3.1157,9.9701,...,0,0,0,0,1,0,0,0,0,0
3,304409,746165,2377316,222.0186,5,5,0,222.0186,4.7891,24.3234,...,0,0,0,0,1,0,0,0,0,0
5,304409,746165,2377317,2.5008,120,120,0,2.5008,0.6184,2.7209,...,0,0,0,0,1,0,0,0,0,0
7,304409,746165,2377318,11.5773,55,55,0,11.5773,0.9060,5.9537,...,0,0,0,0,1,0,0,0,0,0


In [10]:
# summarize the encoded df with the summary statistics
encoded_df.describe()

,ixJobSummary,ixNest,ixPart,dPartTrueArea,cRequired,cNested,fExtShape,dExtArea,dExtBoundaryDist,dExtContainedDist,...,_Strategy_1,_Strategy_10,_Strategy_2,_Strategy_3,_Strategy_4,_Strategy_5,_Strategy_6,_Strategy_7,_Strategy_8,_Strategy_9
count,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,...,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06
mean,1.899243e+06,4.511965e+06,1.580208e+07,-7.351136e+286,1.314087e+01,1.060778e+01,5.439213e+00,6.540658e+02,5.553442e+00,3.577339e+01,...,7.025364e-02,8.826732e-02,1.309068e-02,2.128144e-02,2.868547e-01,3.062653e-02,1.107473e-02,1.056350e-02,1.534973e-02,1.426760e-02
std,1.042945e+06,2.441282e+06,8.736020e+06,inf,2.759251e+03,1.338320e+02,1.045352e+01,1.177717e+04,2.978956e+01,5.134109e+01,...,2.555740e-01,2.836833e-01,1.136632e-01,1.443210e-01,4.522932e-01,1.723037e-01,1.046522e-01,1.022346e-01,1.229395e-01,1.185919e-01
min,3.560100e+04,1.007630e+05,3.047800e+05,-3.742202e+292,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.014986e+06,2.468597e+06,8.436690e+06,3.153750e+01,1.000000e+00,1.000000e+00,0.000000e+00,3.266250e+01,1.898800e+00,9.622400e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.905288e+06,4.519524e+06,1.593877e+07,1.058387e+02,2.000000e+00,2.000000e+00,0.000000e+00,1.105994e+02,3.395500e+00,1.900000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.794505e+06,6.605744e+06,2.329271e+07,4.201447e+02,5.000000e+00,5.000000e+00,5.000000e+00,4.382475e+02,6.750000e+00,4.214070e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,3.688756e+06,8.711716e+06,3.097014e+07,2.960661e+157,5.555555e+06,1.632310e+05,5.400000e+01,1.231630e+07,5.673666e+04,3.239010e+04,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


#### Other data cleaning we might do

In [11]:
# Convert a numeric variable to categorical using custom ranges
# bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
# labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
# df['age_group'] = pd.cut(df['age'], bins=bins, labels=lab

In [12]:
# output the results as a cleaned csv 
encoded_df.to_csv('../CEIP_csv/cleaned.csv', index=False)

#### Downsampling Data
Downsampling data to ~1,000,000 samples for early machine learning & analysis